In [101]:
# Importing some libraries beforehand
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [102]:
# Import the dataset(ds)
ds = pd.read_csv('1655023Pet shop.csv')
ds.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461
3,4,105632,health,off_brand,2.6,8,NaN,5.0,1579
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643


Assignment 1: (15%)

First, turn the categorical variables into dummy variables and explain which category you chose as the reference category. Second, run a regression model where all independent variables are included in a single model. Use Cook’s D to find out if there are any outliers. Note: you will first have to remove missing values first in order to get Cooks D to work.

After you identified the relevant outliers, go back to the original data and turn these outliers into missing values.

In [103]:
# First, turn the categorical variables into dummy variables
product_category_dummies = pd.get_dummies(ds['product_category'])
product_quality_dummies = pd.get_dummies(ds['quality'])
ds_dummies = pd.concat([ds,product_category_dummies,product_quality_dummies], axis=1)
ds_dummies.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852,False,True,False,False,True,False
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045,False,False,True,False,True,False
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461,False,False,False,True,True,False
3,4,105632,health,off_brand,2.6,8,NaN,5.0,1579,False,True,False,False,True,False
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643,False,False,False,True,True,False


In [104]:
# Calculate the number of categories in 'product_category'
category_counts = ds['product_category'].value_counts()

category_counts

product_category
health    858
toys      824
other     296
food      272
Name: count, dtype: int64

In [105]:
# Calculate the number of categories in 'product_category'
quality_counts = ds['quality'].value_counts()

quality_counts

quality
off_brand    1263
premium       876
Name: count, dtype: int64

In my analysis, I chose other and off_brand as my reference categories to compare product_category and quality.
I chose other because it's a mixed category and health and toys were almost the same in numbers. This way, I could easily see how the special types health, toys or food did against a mix of everything else.

I went with off_brand as the standard for quality because we had more of these than premium ones. This lets me show if premium really means more sales compared to the usual off-brand products.

In [106]:
# Check for missing values in the dataset
missing_values = ds_dummies.isnull().sum()
missing_values

Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount              0
retail_price        157
perc_physical        64
market_size           0
food                  0
health                0
other                 0
toys                  0
off_brand             0
premium               0
dtype: int64

In [107]:
#dropping the missing values
ds_clean = ds_dummies.dropna()
ds_clean.isnull().sum()

Unnamed: 0          0
products_sold       0
product_category    0
quality             0
satisfaction        0
discount            0
retail_price        0
perc_physical       0
market_size         0
food                0
health              0
other               0
toys                0
off_brand           0
premium             0
dtype: int64

In [108]:
ds_clean.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852,False,True,False,False,True,False
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045,False,False,True,False,True,False
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461,False,False,False,True,True,False
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643,False,False,False,True,True,False
5,6,55492,other,off_brand,3.6,8,19.0,40.8,915,False,False,True,False,True,False


In [109]:
model1 = smf.ols('products_sold ~ health + toys + food + satisfaction + discount + retail_price + perc_physical + market_size + premium', data=ds_clean).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:          products_sold   R-squared:                       0.783
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     774.5
Date:                Tue, 02 Apr 2024   Prob (F-statistic):               0.00
Time:                        17:48:29   Log-Likelihood:                -22325.
No. Observations:                1942   AIC:                         4.467e+04
Df Residuals:                    1932   BIC:                         4.473e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -1.06e+05   5091.217    -

In [110]:
CooksD = model1.get_influence().cooks_distance
n = len(ds_clean)
ds_clean['outlier'] = CooksD[0] > 4/n
ds_clean.head()

/tmp/ipykernel_54654/229332129.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_clean['outlier'] = CooksD[0] > 4/n


,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,outlier
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852,False,True,False,False,True,False,False
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045,False,False,True,False,True,False,False
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461,False,False,False,True,True,False,False
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643,False,False,False,True,True,False,False
5,6,55492,other,off_brand,3.6,8,19.0,40.8,915,False,False,True,False,True,False,False


In [111]:
print(ds_clean[ds_clean.outlier == True])

      Unnamed: 0  products_sold product_category    quality  satisfaction  \
25            26         146040            other  off_brand           4.8   
93            94          38877             food    premium           3.2   
104          105         103302            other    premium           2.4   
147          148         165368           health  off_brand           3.7   
199          200          54201           health    premium           4.4   
...          ...            ...              ...        ...           ...   
2116        2117          54890           health  off_brand           2.2   
2135        2136         138834            other  off_brand           2.6   
2149        2150         178843            other    premium           4.7   
2154        2155          74674             food    premium           3.1   
2196        2197          95990             toys    premium           2.9   

      discount  retail_price  perc_physical  market_size   food  health  \


After looking at these 83 outliers, there are not much that are impossible except for the discount weeks which were above 52 and the perc_phyical percentages which were above 100. These impossible outliers will be removed to a missing value in the original dataset: ds_dummies.

In [112]:
ds_dummies.loc[ds_dummies['discount']>52,'discount'] = np.nan
ds_dummies.loc[ds_dummies['perc_physical']>100,'perc_physical'] = np.nan
ds_dummies.isnull().sum()

Unnamed: 0            0
products_sold         0
product_category      0
quality             111
satisfaction          0
discount             22
retail_price        157
perc_physical        89
market_size           0
food                  0
health                0
other                 0
toys                  0
off_brand             0
premium               0
dtype: int64

As there can be seen there are some missing values added to discount as well as perc_physical.

Assignment 2: (15%)

The original data contained missing values, and if you did assignment 1 correctly some more should be added. Use the correct imputation techniques for dealing with both the categorical and continuous missing values. Explain what you did. After this, check if there are potential issues with multicollinearity, and if there are, explain how you dealt with it.

In [113]:
ds2 = pd.get_dummies(ds_dummies, dummy_na=True)
ds2.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,...,off_brand,premium,product_category_food,product_category_health,product_category_other,product_category_toys,product_category_nan,quality_off_brand,quality_premium,quality_nan
0,1,67648,3.5,12.0,33.0,42.3,852,False,True,False,...,True,False,False,True,False,False,False,True,False,False
1,2,120421,4.5,10.0,16.0,54.3,1045,False,False,True,...,True,False,False,False,True,False,False,True,False,False
2,3,95242,2.9,6.0,25.0,88.2,1461,False,False,False,...,True,False,False,False,False,True,False,True,False,False
3,4,105632,2.6,8.0,NaN,5.0,1579,False,True,False,...,True,False,False,True,False,False,False,True,False,False
4,5,221627,4.9,11.0,25.0,57.0,2643,False,False,False,...,True,False,False,False,False,True,False,True,False,False


In this step above I transformed my categorical data by making dummy variables and a seperate column for missing values of the quality. Since there were no missing values in the 

You transformed the categorical data in your dataset (ds_dummies) into a format that a regression model can work with. This is done by making dummy variables, which are just columns where the presence of a category is marked with a 1 and its absence with a 0. The dummy_na=True part means you also made a separate column for missing values, where missing data is marked with a 1, letting the model know where the data is missing.
Handled Missing Values:

Next, you addressed the missing values in both your categorical and continuous data. You used an advanced technique called Iterative Imputer, which goes through your data and makes smart guesses about what the missing values could be. It uses all the other information in your dataset to make these guesses as accurate as possible. It's like filling in the blanks by looking at the surrounding words and guessing what should be there. This is done in a loop until the system is confident about its guesses.
Converted Back to DataFrame:

After the missing values were guessed and filled in, you converted this imputed data back into a pandas DataFrame. This is basically just organizing the data back into a table format, which is easier to work with in Python.
Checked for Missing Values Again:

Finally, you checked to make sure that there were no missing values left. This step is like proofreading your work to ensure that all the blanks have been filled in.

In [114]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputed_data = IterativeImputer().fit_transform(ds2)
imputed_data = pd.DataFrame(imputed_data, columns=ds2.columns)
imputed_data.isnull().sum()

Unnamed: 0                 0
products_sold              0
satisfaction               0
discount                   0
retail_price               0
perc_physical              0
market_size                0
food                       0
health                     0
other                      0
toys                       0
off_brand                  0
premium                    0
product_category_food      0
product_category_health    0
product_category_other     0
product_category_toys      0
product_category_nan       0
quality_off_brand          0
quality_premium            0
quality_nan                0
dtype: int64

In [115]:
imputed_data.head()

,Unnamed: 0,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,...,off_brand,premium,product_category_food,product_category_health,product_category_other,product_category_toys,product_category_nan,quality_off_brand,quality_premium,quality_nan
0,1.0,67648.0,3.5,12.0,33.000000,42.3,852.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2.0,120421.0,4.5,10.0,16.000000,54.3,1045.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3.0,95242.0,2.9,6.0,25.000000,88.2,1461.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,4.0,105632.0,2.6,8.0,18.705641,5.0,1579.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5.0,221627.0,4.9,11.0,25.000000,57.0,2643.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [116]:
columns_to_drop = ['Unnamed: 0', 'product_category_food', 'product_category_health', 'product_category_other', 'product_category_toys', 'quality_off_brand', 'quality_premium', 'product_category_nan']
imputed_data = imputed_data.drop(columns=columns_to_drop)
imputed_data.head()

,products_sold,satisfaction,discount,retail_price,perc_physical,market_size,food,health,other,toys,off_brand,premium,quality_nan
0,67648.0,3.5,12.0,33.000000,42.3,852.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,120421.0,4.5,10.0,16.000000,54.3,1045.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,95242.0,2.9,6.0,25.000000,88.2,1461.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,105632.0,2.6,8.0,18.705641,5.0,1579.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,221627.0,4.9,11.0,25.000000,57.0,2643.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
